In [13]:
import numpy as np
import logging
import six
import more_itertools
import glob
import struct
import os
import librosa

# sklearn                                                                                                                                                                             
from sklearn.model_selection import train_test_split

#HYPER PARAMETER
FREQUENCY  =16000


In [ ]:
# Feed-Forward with gated activation hidden units                                                                                                                                     
class MLP(chainer.Chain):
   def __init__(self, n_units):
      self.layer_name = ["l%d" % l for l in range(len(n_units))]
      self.postfix = {l:[".%s.sigm.W" % l, ".%s.sigm.b" % l, \
               ".%s.tanh.W" % l, ".%s.tanh.b" % l] for l in self.layer_name}

      layers = {}
      for l in range(1, len(n_units)):
         layers[self.layer_name[l] + "_sigmoid"] = L.Linear(n_units[l-1], n_units[l])
         layers[self.layer_name[l] + "_tanh"]    = L.Linear(n_units[l-1], n_units[l])

      super(MLP, self).__init__(**layers)

   def __call__(self, x):
      h = x

      for n in self.layer_name[1:]:
         if n is not self.layer_name[-1]:
            hs = getattr(self, "%s_sigmoid" %(n))(h)
            ht = getattr(self, "%s_tanh" %(n))(h)
            h = F.sigmoid(hs) * F.tanh(ht)
         else:
            h = getattr(self, "%s_sigmoid" %(n))(h)

      return h

def chainer_variable(x, gpu = -1):
   if 0 <= gpu:
      y = cuda.to_gpu(x, gpu)
   else:
      y = x
   return chainer.Variable(y)

"""i-vector を読み込む関数"""



def writeivector(adv_x, spknum, spkrid):
  header = []
 # drive_root_dir = "./gdrive/My Drive/voxceb1_ivecs_test"                                                                                                                            
  drive_root_dir = "./voxceb1_ivecs_test"

  for j in range(len(adv_x)):
#    fn = drive_root_dir+"/"+spkrid[spknum]+"_"+str(spknum[j].data)+str(j)+".ivec"                                                                                                    
    fn = drive_root_dir+"/"+spkrid_swap[int(spknum[j].data)]+"-adv"+str(j)+".ivec"

    with open(fn, mode='w') as f:
      for i in adv_x[j]:
        f.write(str(i.data)+"\n")
#    print(fn)                                                                                                                                                                        
    #files.download(fn)                                                                                                                                                               

In [ ]:
"""Fast gradient に基づく adversarial example \\                                                                                                                                      
$x^{(\textrm{adv})} = x + \epsilon * \textrm{sign}(\nabla_x \textrm{loss}(G(x), y))$ \\                                                                                               
> $x$: i-vector \\                                                                                                                                                                    
> $y$: speaker label \\                                                                                                                                                               
> $G(\cdot)$: DNN \\                                                                                                                                                                  
> $loss(\cdot)$: cross entropy loss \\                                                                                                                                                
> $\epsilon$: small number                                                                                                                                                            
"""
def fast_gradient(G, x, y, eps = 0.01):

  loss = F.softmax_cross_entropy(G(x), y)
  loss.backward()
  x_adv = cuda.to_cpu(x.data) + eps * np.sign(cuda.to_cpu(x.grad))

  return loss, x_adv

1.mfcc抽出
2.mfccとspkをそれぞれsplit
3.MLPの改良(mfccで学習するように)
4.trainingとtestをMLPに入力(mfccを用いた学習)
5.chainerをpythonに

In [21]:
def readaudio(audio):
    x, fs = librosa.load(audio, sr=FREQUENCY)
    mfcc = librosa.feature.mfcc(y=x, sr=FREQUENCY, n_mfcc=20)
    
    return mfcc

In [49]:
####データ(音声)の読み込み、mfccを抽出####
ROOT="/home/Yuunin/SV/id10270"
MFCC = []
speaker = "Kitt"
for filename in os.listdir(ROOT):
    for utter in os.listdir(ROOT + "/" + filename):
        if ".wav" in utter:
            print(filename + utter)
            mfcc = readaudio(ROOT + "/" + filename + "/" + utter).flatten()
            
            MFCC.append(mfcc)
#  i, s = readaudio(audio)
#  ivec.append(i)
#  spkr.append(s)

5sJomL_D0_g00002.wav
5sJomL_D0_g00001.wav
8jEAjG6SegY00027.wav
8jEAjG6SegY00030.wav
8jEAjG6SegY00004.wav
8jEAjG6SegY00024.wav
8jEAjG6SegY00002.wav
8jEAjG6SegY00036.wav
8jEAjG6SegY00023.wav
8jEAjG6SegY00038.wav
8jEAjG6SegY00010.wav
8jEAjG6SegY00018.wav
8jEAjG6SegY00037.wav
8jEAjG6SegY00011.wav
8jEAjG6SegY00035.wav
8jEAjG6SegY00009.wav
8jEAjG6SegY00006.wav
8jEAjG6SegY00013.wav
8jEAjG6SegY00034.wav
8jEAjG6SegY00016.wav
8jEAjG6SegY00005.wav
8jEAjG6SegY00032.wav
8jEAjG6SegY00019.wav
8jEAjG6SegY00001.wav
8jEAjG6SegY00028.wav
8jEAjG6SegY00008.wav
8jEAjG6SegY00020.wav
8jEAjG6SegY00025.wav
8jEAjG6SegY00021.wav
8jEAjG6SegY00014.wav
8jEAjG6SegY00029.wav
8jEAjG6SegY00026.wav
8jEAjG6SegY00031.wav
8jEAjG6SegY00022.wav
8jEAjG6SegY00033.wav
8jEAjG6SegY00017.wav
8jEAjG6SegY00015.wav
8jEAjG6SegY00003.wav
8jEAjG6SegY00007.wav
8jEAjG6SegY00012.wav
zjwijMp0Qyw00002.wav
zjwijMp0Qyw00001.wav
zjwijMp0Qyw00003.wav
5r0dWxy17C800027.wav
5r0dWxy17C800004.wav
5r0dWxy17C800024.wav
5r0dWxy17C800002.wav
5r0dWxy17C800

In [55]:
print(len(MFCC[157]))

3860


set()重複をなくす、sortにする
list()でリストにする(可変、順番不動なときもある)
sorted()でまたsortする
enumerate()でcountする

In [56]:
#spkrid = { s:i for (i, s) in enumerate(sorted(list(set(spkr)))) } #iは何番目(数字), sは名前(文字)
x = np.asarray(MFCC, dtype=np.float32)
#y = np.asarray([spkrid[s] for s in spkr], dtype=np.int16)
y = 10270
x_train, x_test = train_test_split(x, test_size = 0.1)

print(len(ivec[0]))

print("Data info.")
print("   %d speakers" % len(spkrid.keys()))
print("   training data: %d" % (x_train.shape[0]))
print("   test data: %d" % (x_test.shape[0]))

#spkrid_swap = {v: k for k, v in spkrid.items()} #ちょっと調べる

ValueError: setting an array element with a sequence.

In [36]:
"""ミニバッチ学習 \\                                                                                                                                                                  
$loss = loss_{std} + loss_{adv}$                                                                                                                                                      
> $loss_{std}$: standard cross entropy loss using $x$, $y$ \\                                                                                                                         
> $loss_{adv}$: adversarial loss using $x^{\textrm{(adv)}}$, $y$ \\                                                                                                                   
> $x$: i-vector \\                                                                                                                                                                    
> $y$: speaker label                                                                                                                                                                  
"""

batchsize = 300 # minibatch size                                                                                                                                                      
n_epoch = 200 # number of iterations                                                                                                                                                  
n_hid_units = [256, 256, 512] # units of hidden layers                                                                                                                                
eps = 0.00001 # epsilon for adversarial training                                                                                                                                      
lr = 0.001 # learning rate                                                                                                                                                            
gpu_id = 0 # GPU ID (0)                                                                                                                                                               

# logger                                                                                                                                                                              
logging.basicConfig(format="%(message)s", level=logging.DEBUG) #ここ調べる
logger = logging.getLogger(__name__) #

# GPU setup                                                                                                                                                                           
cuda.get_device(gpu_id).use()

# DNN (adversarial training)                                                                                                                                                          
G_adv = MLP([x_train.shape[1]] + n_hid_units + [np.max(y_train) + 1]) #入力形式を変える
G_adv.to_gpu()

# DNN (standard training)                                                                                                                                                             
G_std = MLP([x_train.shape[1]] + n_hid_units + [np.max(y_train) + 1]) #入力形式を変える
G_std.to_gpu()

# optimizer (adversarial training)                                                                                                                                                    
optimizer_adv = chainer.optimizers.AdaGrad(lr = lr) #chainer to python 
optimizer_adv.setup(G_adv)

# optimizer (adversarial training)                                                                                                                                                    
optimizer_std = chainer.optimizers.AdaGrad(lr = lr) #
optimizer_std.setup(G_std)

# adversarial example generator                                                                                                                                                       
attack = fast_gradient


# minibatch training                                                                                                                                                                  
for epoch in range(n_epoch):
  perm = np.random.permutation(x_train.shape[0])
  n_train = 0
  loss_adv_train, loss_std_train = 0., 0.

  # training                                                                                                                                                                          
  for i in six.moves.range(0, x_train.shape[0], batchsize):
    x = chainer_variable(x_train[perm[i:i+batchsize], :], gpu_id)
    y = chainer_variable(y_train[perm[i:i+batchsize]], gpu_id)

    G_adv.cleargrads()
    G_std.cleargrads()

    # standard training                                                                                                                                                               
    loss = F.softmax_cross_entropy(G_std(x), y)
    loss.backward()
    optimizer_std.update()
    loss_std_train += float(loss.data) * batchsize

    # adversarial training
    loss_std, x_adv2, loss_std_std, x_adv_std = attack(G_adv, x, y, eps)
    x_adv3 = x_adv2.array
    x_adv = x_adv3.astype(np.float32)
    x_adv = chainer_variable(x_adv3, gpu_id)
    loss_adv = F.softmax_cross_entropy(G_adv(x_adv), y)

    G_adv.cleargrads()
    #loss = loss_std + loss_adv                                                                                                                                                       
    loss = loss_adv
    loss.backward()
    optimizer_adv.update()
    loss_adv_train += float(loss.data) * batchsize

    n_train += batchsize

  loss_adv_train /= float(n_train)
  loss_std_train /= float(n_train)

  # evaluation                                                                                                                                                                        
  x = chainer_variable(x_test, gpu_id)
  y = chainer_variable(y_test, gpu_id)

  loss_adv_test, x_adv, loss_std_std, x_adv_std  = attack(G_adv, x, y)
  loss_adv_test_adv = F.softmax_cross_entropy(G_adv(x_adv), y)

  loss_std_test, x_adv2, loss_std_std, x_adv_std = attack(G_std, x, y)
  x_adv3 = x_adv2.array
  x_adv = x_adv3.astype(np.float32)
#  x_adv = chainer.cuda.to_cpu(x_adv2)                                                                                                                                                
  x_adv = chainer_variable(x_adv, gpu_id)
  loss_std_test_adv = F.softmax_cross_entropy(G_std(x_adv), y)

if (epoch + 1) % 10 == 0:
    logger.info("%3d-th epoch (adv - std): %1.3f - %1.3f (train), %1.3f - %1.3f (test), %1.3f - %1.3f (test-adv)" %
         (epoch + 1, loss_adv_train, loss_std_train, \
          float(loss_adv_test.data), float(loss_std_test.data), \
          float(loss_adv_test_adv.data), float(loss_std_test_adv.data)))

if epoch + 1 == n_epoch:
    writeivector(x_adv, y, spkrid_swap)

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.